In [30]:
def H(ra, dec, z):

    ra  = np.array(ra,  dtype=np.float32) 
    dec = np.array(dec, dtype=np.float32) 
    z   = np.array(z,   dtype=np.float32) 

    r = np.float32(cosmo.comoving_distance(z).value)
    x = r*np.cos(np.deg2rad(ra))*np.cos(np.deg2rad(dec))
    y = r*np.sin(np.deg2rad(ra))*np.cos(np.deg2rad(dec))
    z = r*np.sin(np.deg2rad(dec))
    return x, y, z

def get_xi3d(cat_sdss,cat_random,rr):

    # TreeCorr Catalogues
    dd = treecorr.NNCorrelation(config)
    dr = treecorr.NNCorrelation(config)

    gcat = treecorr.Catalog(x=cat_sdss['x'], y=cat_sdss['y'], z=cat_sdss['z'])#, npatch=npatch)

    dd.process(gcat)
    #dr.process(gcat,rcat)

    dd.write('2dcorrelation.out',rr=rr)

    #xi, varxi = dd.calculateXi(rr=rr,dr=dr)
    #r = dd.meanr
    #return xi, varxi, r

import pandas as pd
import numpy as np
import treecorr
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=67.74, Om0=0.3089)
######################
minsep = 10.      # Min theta
maxsep = 80.      # Max theta
nbins = 10        # Bins in theta
#nbootstrap = 500  # No. of bootstrap resampling
#brute = False     # Brute force for TreeCorr
#npatch = 10
nthreads = 4

config = {"min_sep": minsep, \
            "max_sep": maxsep, \
            "nbins": nbins, \
            #"sep_units": 'degree', \
            "bin_type": 'Linear', \
            #"brute": brute, \
            "metric": 'Rperp', \
            #"var_method": 'jackknife', \
            #"num_bootstrap": nbootstrap,\
            "num_threads": nthreads
            }
######################

# Read Gxs
zmin, zmax = 0.01, 0.15
cat_sdss = pd.read_csv('../data/sdss_zmin_0.000_zmax_0.300_sigma_5.0.csv')
cat_sdss  = cat_sdss[(cat_sdss["red"] > zmin)*(cat_sdss["red"] < zmax)]

# Filament Gxs
filgxs = cat_sdss[cat_sdss['dist_fil']<=3.]

# Non-filament Gxs
nonfilgxs = cat_sdss[cat_sdss['dist_fil']>8.25]

# Read Randoms & Build Catalog
cat_random = pd.read_csv('../data/random_sample_healpy_128_561286.csv')
cat_random["x"], cat_random["y"], cat_random["z"] = H(cat_random["ra"], cat_random["dec"], cat_random["red"])
rr = treecorr.NNCorrelation(config)
rcat = treecorr.Catalog(x=cat_random['x'], y=cat_random['y'], z=cat_random['z'])#, npatch=npatch)
rr.process(rcat)

get_xi3d(cat_sdss,cat_random,rr)

#xi_filgxs, varxi_filgxs, r = get_xi3d(filgxs,cat_random)

#xi_nonfilgxs, varxi_nonfilgxs, r = get_xi3d(nonfilgxs,cat_random)


In [31]:
filgxs = pd.read_csv('2dcorrelation.out',sep='\s+')

print(filgxs.head)

<bound method NDFrame.head of             ## {'min_sep':       10.0,   'max_sep':       80.0,    'nbins':  \
0            #       r_nom       meanr     meanlogr          xi    sigma_xi   
1   1.3500e+01  1.3752e+01  2.6109e+00  -7.2546e-02  8.3127e-05  3.0657e+08   
2   2.0500e+01  2.0673e+01  3.0244e+00  -6.8575e-02  6.8938e-05  4.4766e+08   
3   2.7500e+01  2.7626e+01  3.3163e+00  -6.2847e-02  6.0585e-05  5.8316e+08   
4   3.4500e+01  3.4597e+01  3.5422e+00  -5.6746e-02  5.5053e-05  7.1085e+08   
5   4.1500e+01  4.1578e+01  3.7265e+00  -4.9204e-02  5.1094e-05  8.3190e+08   
6   4.8500e+01  4.8564e+01  3.8821e+00  -4.2549e-02  4.8113e-05  9.4474e+08   
7   5.5500e+01  5.5551e+01  4.0167e+00  -3.9245e-02  4.5791e-05  1.0466e+09   
8   6.2500e+01  6.2543e+01  4.1354e+00  -3.6598e-02  4.3946e-05  1.1394e+09   
9   6.9500e+01  6.9536e+01  4.2415e+00  -3.3561e-02  4.2469e-05  1.2239e+09   
10  7.6500e+01  7.6529e+01  4.3374e+00  -3.0761e-02  4.1284e-05  1.2989e+09   

           10, 'bin_t

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_120532/2446443699.py:1: SyntaxWarning: invalid escape sequence '\s'
  filgxs = pd.read_csv('2dcorrelation.out',sep='\s+')


In [32]:
import pandas as pd
import numpy as np

# Load the TreeCorr output file
data = pd.read_csv("2dcorrelation.out", delim_whitespace=True, comment='#')

# Extract the relevant columns
sigma = data['sigma']
pi = data['pi']
xi = data['xi']
npairs = data['npairs']  # Number of pairs in each bin (for weighting)

# Compute xi_perp: Group by sigma and average over all pi bins
xi_perp = data.groupby('sigma').apply(
    lambda group: np.average(group['xi'], weights=group['npairs'])
).reset_index(name='xi_perp')

# Compute xi_parallel: Group by pi and average over all sigma bins
xi_parallel = data.groupby('pi').apply(
    lambda group: np.average(group['xi'], weights=group['npairs'])
).reset_index(name='xi_parallel')

# Save to files
xi_perp.to_csv("xi_perp.csv", index=False)
xi_parallel.to_csv("xi_parallel.csv", index=False)


/tmp/ipykernel_120532/1932557408.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv("2dcorrelation.out", delim_whitespace=True, comment='#')


KeyError: 'sigma'